In [68]:
import sys
sys.path.append("../")

In [69]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [70]:
df_raw = pd.read_pickle("../data/EUR_USD_H1.pkl")

In [71]:
df_raw.shape

(6142, 14)

In [73]:
df_an = df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [74]:
df_an.shape

(6000, 14)

In [75]:
df_an = RSI(df_an)

In [76]:
df_an.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
5995,2021-12-30 19:00:00+00:00,1063,1.13058,1.13298,1.13043,1.13274,1.13052,1.13291,1.13036,1.13267,1.13065,1.13306,1.13049,1.13280,51.407635
5996,2021-12-30 20:00:00+00:00,1092,1.13272,1.13304,1.13185,1.13193,1.13264,1.13297,1.13177,1.13186,1.13279,1.13311,1.13192,1.13200,48.671929
5997,2021-12-30 21:00:00+00:00,388,1.13195,1.13276,1.13194,1.13258,1.13188,1.13269,1.13187,1.13249,1.13202,1.13283,1.13201,1.13267,50.928675
5998,2021-12-30 22:00:00+00:00,118,1.13252,1.13264,1.13222,1.13230,1.13204,1.13236,1.13182,1.13218,1.13299,1.13299,1.13236,1.13243,49.910668
5999,2021-12-30 23:00:00+00:00,507,1.13227,1.13258,1.13190,1.13224,1.13214,1.13251,1.13183,1.13216,1.13240,1.13266,1.13198,1.13231,49.681497


In [77]:
df_an = apply_patterns(df_an)

In [78]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [79]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [80]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
                'ask_c','bid_c', 'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]

In [81]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [82]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14
0,2021-01-27 05:00:00+00:00,1.21573,1.21652,1.21562,1.21632,1.21641,1.21622,False,1,1.214219,56.208275
1,2021-01-27 06:00:00+00:00,1.21630,1.21660,1.21588,1.21631,1.21638,1.21624,False,1,1.214243,56.124731
2,2021-01-27 07:00:00+00:00,1.21632,1.21632,1.21518,1.21552,1.21559,1.21545,True,-1,1.214258,49.824341
3,2021-01-27 08:00:00+00:00,1.21554,1.21586,1.21439,1.21482,1.21488,1.21475,False,-1,1.214264,45.003578
4,2021-01-27 09:00:00+00:00,1.21480,1.21489,1.21194,1.21228,1.21234,1.21221,False,-1,1.214242,32.656498


### start test the strategy

In [83]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE        

In [84]:
df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [85]:
df_slim["SIGNAL"].value_counts()

 0    5262
-1     332
 1     207
Name: SIGNAL, dtype: int64

In [86]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5

def apply_take_profit(row):
    if row.SIGNAL != NONE:
        return (row.mid_c - row.mid_o) * 1.5 + row.mid_c
    else:
        return 0.0

def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else:
        return 0.0

In [87]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [88]:
df_slim[df_slim.SIGNAL==BUY].head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL
216,2021-02-09 05:00:00+00:00,1.20760,1.20808,1.20760,1.20781,1.20788,1.20774,True,1,1.205156,71.584393,1,1.208125,1.20760
218,2021-02-09 07:00:00+00:00,1.20773,1.20870,1.20768,1.20814,1.20820,1.20808,True,1,1.205212,72.461530,1,1.208755,1.20773
248,2021-02-10 13:00:00+00:00,1.21166,1.21398,1.21144,1.21286,1.21292,1.21280,True,1,1.206890,59.927244,1,1.214660,1.21166
250,2021-02-10 15:00:00+00:00,1.21244,1.21396,1.21232,1.21311,1.21317,1.21305,True,1,1.207008,60.336695,1,1.214115,1.21244
279,2021-02-11 20:00:00+00:00,1.21307,1.21375,1.21297,1.21328,1.21334,1.21322,True,1,1.208452,55.502223,1,1.213595,1.21307


In [89]:
df_plot = df_slim.iloc[70:100]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]

markers = ['mid_c', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker=dict(color=marker_colors[i], size=12)
    ))

cp.show_plot(line_traces=["EMA_200"], sec_traces=['RSI_14'], height=250)

In [90]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.trigger_price = row.mid_c
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)    

In [91]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]
    
    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))    

In [92]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [93]:
df_results.result.sum()

33.5

### merge H1 and M5

In [94]:
df_results.sort_values(by="start_index", inplace=True)

In [95]:
df_m5 = pd.read_pickle("../data/EUR_USD_M5.pkl")

In [96]:
df_m5.shape

(73354, 14)

In [97]:
df_m5.time.max()

Timestamp('2021-12-30 23:55:00+0000', tz='tzutc()')

In [98]:
df_raw.time.max()

Timestamp('2021-12-30 23:00:00+0000', tz='tzutc()')

In [99]:
from dateutil import parser

In [100]:
time_min = parser.parse("2021-12-15T10:00:00Z")
time_max = parser.parse("2021-12-15T11:00:00Z")
df_m5_s = df_m5[(df_m5.time>=time_min)&(df_m5.time<=time_max)]
df_raw_s = df_raw[(df_raw.time>=time_min)&(df_raw.time<=time_max)]

In [101]:
df_m5_s

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
70035,2021-12-15 10:00:00+00:00,208,1.12716,1.12745,1.12709,1.12734,1.12708,1.12738,1.12702,1.12727,1.12723,1.12752,1.12715,1.12742
70036,2021-12-15 10:05:00+00:00,132,1.12733,1.12754,1.12726,1.12746,1.12726,1.12747,1.12719,1.12739,1.12740,1.12760,1.12733,1.12752
70037,2021-12-15 10:10:00+00:00,166,1.12744,1.12752,1.12732,1.12746,1.12737,1.12745,1.12725,1.12739,1.12750,1.12760,1.12739,1.12753
70038,2021-12-15 10:15:00+00:00,138,1.12744,1.12770,1.12740,1.12761,1.12737,1.12763,1.12733,1.12754,1.12752,1.12776,1.12747,1.12768
70039,2021-12-15 10:20:00+00:00,252,1.12760,1.12762,1.12712,1.12712,1.12753,1.12756,1.12705,1.12705,1.12767,1.12769,1.12719,1.12719
70040,2021-12-15 10:25:00+00:00,198,1.12711,1.12726,1.12695,1.12695,1.12704,1.12719,1.12688,1.12688,1.12718,1.12733,1.12702,1.12702
70041,2021-12-15 10:30:00+00:00,142,1.12696,1.12698,1.12680,1.12696,1.12689,1.12691,1.12674,1.12690,1.12703,1.12706,1.12686,1.12703
70042,2021-12-15 10:35:00+00:00,121,1.12698,1.12712,1.12698,1.12700,1.12691,1.12705,1.12691,1.12693,1.12704,1.12718,1.12704,1.12706
70043,2021-12-15 10:40:00+00:00,206,1.12698,1.12707,1.12651,1.12656,1.12692,1.12700,1.12644,1.12649,1.12705,1.12714,1.12658,1.12663
70044,2021-12-15 10:45:00+00:00,165,1.12655,1.12668,1.12645,1.12654,1.12648,1.12661,1.12638,1.12648,1.12662,1.12675,1.12652,1.12661


In [102]:
df_raw_s

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
5864,2021-12-15 10:00:00+00:00,2038,1.12716,1.1277,1.12634,1.12673,1.12708,1.12763,1.12627,1.12666,1.12723,1.12776,1.12640,1.12680
5865,2021-12-15 11:00:00+00:00,2138,1.12674,1.1270,1.12626,1.12698,1.12667,1.12693,1.12619,1.12691,1.12681,1.12706,1.12631,1.12705


In [103]:
df_m5_slim = df_m5[['time','mid_h', 'mid_l' ]].copy()

In [104]:
df_m5_slim.head()

,time,mid_h,mid_l
0,2021-01-07 00:00:00+00:00,1.23390,1.23332
1,2021-01-07 00:05:00+00:00,1.23379,1.23340
2,2021-01-07 00:10:00+00:00,1.23408,1.23366
3,2021-01-07 00:15:00+00:00,1.23444,1.23398
4,2021-01-07 00:20:00+00:00,1.23432,1.23382


In [105]:
df_signals = df_slim[df_slim.SIGNAL != NONE].copy() 

In [106]:
df_signals['m5_start'] = [x + dt.timedelta(hours=1) for x in df_signals.time]
# if buy at 10 am according to H1 candle, need the 5M candle in next hour

In [107]:
df_signals['start_index_h1'] = df_signals.index

In [108]:
df_signals.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL,m5_start,start_index_h1
6,2021-01-27 11:00:00+00:00,1.21311,1.21312,1.21135,1.21166,1.21173,1.21159,True,-1,1.214199,33.130880,-1,1.209485,1.21311,2021-01-27 12:00:00+00:00,6
9,2021-01-27 14:00:00+00:00,1.21113,1.21125,1.20585,1.20684,1.20690,1.20677,True,-1,1.214043,22.321149,-1,1.200405,1.21113,2021-01-27 15:00:00+00:00,9
14,2021-01-27 19:00:00+00:00,1.21202,1.21290,1.20980,1.21001,1.21008,1.20994,True,-1,1.213853,40.266493,-1,1.206995,1.21202,2021-01-27 20:00:00+00:00,14
30,2021-01-28 11:00:00+00:00,1.21088,1.21116,1.20959,1.21023,1.21030,1.21016,True,-1,1.213232,46.831223,-1,1.209255,1.21088,2021-01-28 12:00:00+00:00,30
45,2021-01-29 02:00:00+00:00,1.21086,1.21092,1.21022,1.21044,1.21051,1.21037,True,-1,1.213045,43.060255,-1,1.209810,1.21086,2021-01-29 03:00:00+00:00,45


In [109]:
df_signals.columns

Index(['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c', 'bid_c',
       'ENGULFING', 'direction', 'EMA_200', 'RSI_14', 'SIGNAL', 'TP', 'SL',
       'm5_start', 'start_index_h1'],
      dtype='object')

In [110]:
df_signals.drop(['time', 'mid_o', 'mid_h', 'mid_l', 'ask_c', 'bid_c',
                'ENGULFING', 'EMA_200', 'RSI_14', 'direction'], axis=1, inplace=True)

In [111]:
df_signals.head()

,mid_c,SIGNAL,TP,SL,m5_start,start_index_h1
6,1.21166,-1,1.209485,1.21311,2021-01-27 12:00:00+00:00,6
9,1.20684,-1,1.200405,1.21113,2021-01-27 15:00:00+00:00,9
14,1.21001,-1,1.206995,1.21202,2021-01-27 20:00:00+00:00,14
30,1.21023,-1,1.209255,1.21088,2021-01-28 12:00:00+00:00,30
45,1.21044,-1,1.209810,1.21086,2021-01-29 03:00:00+00:00,45


In [112]:
df_signals.rename(columns={
    'mid_c' : 'start_price',
    'm5_start' : 'time'
}, inplace=True)

In [113]:
df_signals.head(2)

,start_price,SIGNAL,TP,SL,time,start_index_h1
6,1.21166,-1,1.209485,1.21311,2021-01-27 12:00:00+00:00,6
9,1.20684,-1,1.200405,1.21113,2021-01-27 15:00:00+00:00,9


In [114]:
df_m5_slim.head(2)

,time,mid_h,mid_l
0,2021-01-07 00:00:00+00:00,1.23390,1.23332
1,2021-01-07 00:05:00+00:00,1.23379,1.23340


In [120]:
merged = pd.merge(left=df_m5_slim, right=df_signals, on='time', how='left')

In [121]:
merged.fillna(0, inplace=True)

In [122]:
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)

In [123]:
merged.head()

,time,mid_h,mid_l,start_price,SIGNAL,TP,SL,start_index_h1
0,2021-01-07 00:00:00+00:00,1.23390,1.23332,0.0,0,0.0,0.0,0
1,2021-01-07 00:05:00+00:00,1.23379,1.23340,0.0,0,0.0,0.0,0
2,2021-01-07 00:10:00+00:00,1.23408,1.23366,0.0,0,0.0,0.0,0
3,2021-01-07 00:15:00+00:00,1.23444,1.23398,0.0,0,0.0,0.0,0
4,2021-01-07 00:20:00+00:00,1.23432,1.23382,0.0,0,0.0,0.0,0


In [124]:
class TradeM5:
    def __init__(self, row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1 = row.start_index_h1
        self.start_price = row.start_price
        self.trigger_price = row.start_price
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 1
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)   

In [125]:
open_trades_m5 = []
closed_trades_m5 = []

for index, row in merged.iterrows():
    for ot in open_trades_m5:
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
    open_trades_m5 = [x for x in open_trades_m5 if x.running == True]
    
    if row.SIGNAL != NONE:
        open_trades_m5.append(TradeM5(row))   

In [126]:
df_res_m5 = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5])

In [127]:
df_res_m5.head()

,running,start_index_m5,start_index_h1,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,duration
0,False,4157,6,1.21166,1.20898,-1,1.209485,1.21311,1.5,2021-01-27 14:30:00+00:00,2021-01-27 12:00:00+00:00,31
1,False,4193,9,1.20684,1.21156,-1,1.200405,1.21113,-1.0,2021-01-27 17:40:00+00:00,2021-01-27 15:00:00+00:00,33
2,False,4444,30,1.21023,1.21103,-1,1.209255,1.21088,-1.0,2021-01-28 12:10:00+00:00,2021-01-28 12:00:00+00:00,3
3,False,4253,14,1.21001,1.21309,-1,1.206995,1.21202,-1.0,2021-01-28 13:55:00+00:00,2021-01-27 20:00:00+00:00,215
4,False,4624,45,1.21044,1.20980,-1,1.209810,1.21086,1.5,2021-01-29 03:40:00+00:00,2021-01-29 03:00:00+00:00,9


In [128]:
df_res_m5.result.sum()

-16.0